## Day 21

In [1]:
using BenchmarkTools
function read21()
    lines = split(strip(read(joinpath(@__DIR__, "./input/day21.txt"), String)), '\n')
    ingredients= getproperty.(match.(r"(.*?) \(contains (.*)\)", lines), :captures)
    ingredients = map(lines) do l
        m = match(r"(.*?) \(contains (.*)\)",l)
        split(m[1], " ")
    end
    allergens = map(lines) do l
        m = match(r"(.*?) \(contains (.*)\)",l)
        split(m[2], ", ")
    end
    return ingredients, allergens
end

read21 (generic function with 1 method)

In [2]:
function day21_part1(data = read21())
   ingredients, allergens = data
   all_allergens = unique(vcat(allergens...))
   d = Dict()
   while length(d) != length(all_allergens) 
        # loop through all allergens that have not been identified
       for allergen in setdiff(all_allergens, keys(d))
            overlap = intersect(ingredients[allergen .∈ allergens]...)
            if length(overlap) ==  1 
                d[allergen] = overlap  # save
                # remove from ingredients
                ingredients = setdiff.(ingredients, [overlap])
            end
       end
    end
        
    # part2
    order = sortperm(vcat(keys(d)...))
    return length(vcat(ingredients...)),  join(vcat(values(d)...)[order],",")
end

day21_part1 (generic function with 2 methods)

In [3]:
@btime day21_part1()

  3.173 ms (7738 allocations: 3.61 MiB)


(2061, "cdqvp,dglm,zhqjs,rbpg,xvtrfz,tgmzqjz,mfqgx,rffqhl")

## Day 22

In [4]:
using BenchmarkTools
function read22()
    decks = split(strip(read(joinpath(@__DIR__, "./input/day22.txt"), String)), "\n\n")
    p1 = parse.(Int, split(decks[1], "\n")[2:end])
    p2 = parse.(Int, split(decks[2], "\n")[2:end])
    return p1, p2
end

read22 (generic function with 1 method)

In [5]:
function day22_part1(data=read22())
   p1, p2 = data
   
   while length(p1) > 0 && length(p2) > 0
        p1, p2 = play_round(p1, p2)
    end
    winner = length(p1) > 0 ? p1 : p2
    
    return sum(winner .* collect(length(winner):-1:1))
end

function play_round(p1, p2)
   if p1[1] > p2[1]
        # p1 wins
        p1 = vcat(p1[2:end], p1[1], p2[1])
        p2 = p2[2:end]
    elseif p1[1] < p2[1]
        # p2 wins
        p2 = vcat(p2[2:end], p2[1], p1[1])
        p1 = p1[2:end]
    end
    return p1, p2
end

play_round (generic function with 1 method)

In [6]:
@btime day22_part1()

  565.721 μs (7981 allocations: 561.38 KiB)


33631

In [7]:
function day22_part2(data = read22())
    p1, p2 = data
    
    w, p = play_game(p1, p2)
    
    return sum(p .* collect(length(p):-1:1))
end

function play_game(p1, p2)
    seen = Set{Tuple{Array,Array}}([])
    
    while length(p1) > 0 && length(p2) > 0
        (p1, p2) ∈ seen && return 1, p1  # infinite-game-prevention rule
        push!(seen, copy.((p1, p2)))
        
        c1, c2 = popfirst!.((p1, p2))
        
        if c1 <= length(p1) && c2 <= length(p2)
            #play subgame
            w, _  = play_game(p1[1:c1], p2[1:c2])
        elseif c1 > c2 w = 1
        elseif c2 > c1 w = 2
        else @error "Unexpected case: c₁ == c₂" end
        
        w == 1 && push!(p1, c1, c2)
        w == 2 && push!(p2, c2, c1)
        
        
    end
    
    return  length(p1) > 0 ? (1, p1) : (2, p2)
end

play_game (generic function with 1 method)

In [8]:
@btime day22_part2()

  3.056 s (11505409 allocations: 1.00 GiB)


33469

## Day 23

In [9]:
using BenchmarkTools
read23 = function(data="156794823")
   parse.(Int,split(data, "") )
end

#8 (generic function with 2 methods)

In [10]:
function day23_part1(data=read23())
    data = crabcups(data)
    
    # find 1 
    pos = findfirst(x->x==1, data)
    data = circshift(data, 1-pos)
    return join(data[2:end])
end

function crabcups(data::Array{Int64,1}; steps=100)
    size = length(data)
    
    pop = similar(data, 3)
    
    
    for i in 1:steps
        current = data[1]
        pop = data[2:4]
        #data = data[[1, 5:end...]]

        dest = mod1(current-1, size)
        while dest in pop
            dest = mod1(dest-1, size)
        end

        dest_pos=findfirst(x->x==dest, data)
        data = data[[1, 5:dest_pos..., 2:4..., dest_pos+1:end...]]
        data = circshift(data, -1)
    end  
    return data
end

crabcups (generic function with 1 method)

In [11]:
@btime day23_part1()

  78.223 μs (1548 allocations: 94.16 KiB)


"82573496"

In [12]:
function mypeek(next, at, n; result=similar(next,n))
    for i in 1:n
        result[i] = next[at]
        at = next[at]
    end
    result
end

# https://blog.kdheepak.com/advent-of-code-2020-retrospective.html#day-23-crab-cups
function crabcups2(cups; steps=100)
    N = length(cups)
    prealloc = similar(cups, 3)

    next = similar(cups)
    for i in 1:N
        next[cups[i]] = cups[mod1(i+1,N)]
    end

    current = cups[1]
    for i in 1:steps

        pickups = mypeek(next, current, 3, result=prealloc)

        dst = mod1(current-1, N)
        while dst in pickups
          dst = mod1(dst-1, N)
        end

        next[current] = next[pickups[end]]  ## das nächste nach current wird das nächste nach dem ende von pickup
        next[pickups[end]] = next[dst] # das nächste nach dem ende von pickup wird das ende nach dem ziel
        next[dst] = pickups[1] # nach dem ziel kommt der anfang von pickup
        current = next[current] #eine position weiter
           
    end

    return next
end

crabcups2 (generic function with 1 method)

In [13]:
function day23_part1b(data=read23())
    data = crabcups2(data, steps=100)
    return join(mypeek(data, 1, 8))
end

function day23_part2(data=read23())
    
    data = vcat(data, [10:1000000]...)
    
    data = crabcups2(data, steps=10000000)
    
    
    return prod(mypeek(data, 1, 2))
end

day23_part2 (generic function with 2 methods)

In [14]:
@btime day23_part2()

  680.180 ms (40 allocations: 15.26 MiB)


11498506800

In [15]:
@btime day23_part1b()

  2.476 μs (48 allocations: 3.53 KiB)


"82573496"

## Day 24

In [16]:
using BenchmarkTools
function read24()
   return split(strip(read(joinpath(@__DIR__, "./input/day24.txt"), String)), "\n")
end

read24 (generic function with 1 method)

In [17]:
function hex_grid_pos(move)
   x = 0
   y = 0
   while(!isempty(move))
    if startswith(move, "e")
        x = x -2
        move = move[2:end]
    elseif startswith(move, "w")
        x = x +2
        move = move[2:end]
    elseif startswith(move, "ne")
        x = x -1
        y = y+1
        move = move[3:end]
    elseif startswith(move, "nw")
        x = x +1
        y = y+1
        move = move[3:end]
    elseif startswith(move, "se")
        x = x -1
        y = y-1
        move = move[3:end]
    elseif startswith(move, "sw")
        x = x +1
        y = y-1
        move = move[3:end]
     end
   end
    return x,y
        
end

hex_grid_pos (generic function with 1 method)

In [18]:
function day24_part1(data=read24())
   tiles = []
    
    for move in data
       x,y =  hex_grid_pos(move)
        if [x,y] in tiles
           setdiff!(tiles, [[x,y]])
        else
           append!(tiles, [[x,y]])
        end
    end
    return tiles
end

day24_part1 (generic function with 2 methods)

In [19]:
@btime length(day24_part1())

  3.490 ms (15727 allocations: 646.91 KiB)


263

In [20]:
function day24_part2()
   tiles = day24_part1()
    
   for i in 1:100
        neighbors = black_neighbors(tiles)
        new_tiles = []
        for (pos, n) in neighbors
           if pos in tiles  ## currently black
               if 1 <= n <= 2
                    append!(new_tiles, [pos])
                    ## keep black
                end
            else  # currently white
               if n == 2 
                   append!(new_tiles, [pos]) 
               end
            end
        end
        tiles = new_tiles
    end
    
    return length(tiles)
end


function black_neighbors(tiles)
   n = Dict()
    directions = [[-2,0], [2,0], [-1,1], [1,1], [-1,-1], [1,-1]]
   for pos in tiles
        for d in directions
            if haskey(n, pos+d)
                n[pos+d] =  n[pos+d]+1
            else
                n[pos+d] = 1
            end
        end
   end
    return n
end

black_neighbors (generic function with 1 method)

In [21]:
@btime day24_part2()

  20.326 s (2951218 allocations: 262.65 MiB)


3649

## Day 25

In [22]:
function read25()
    return parse.(Int, split(strip(read(joinpath(@__DIR__, "./input/day25.txt"), String)), "\n"))
end

read25 (generic function with 1 method)

In [23]:
function day25_part1(data=read25())
    public_door, public_card = data[1], data[2]
    
    display("Pub door: $public_door, card: $public_card")
    
    loop_door = 0
    while powermod(7, loop_door, 20201227) != public_door
       loop_door += 1
    end
    
    loop_card = 0
    while powermod(7, loop_card, 20201227) != public_card
       loop_card += 1
    end
   
    display("loop door $loop_door, card $loop_card")
    
    return powermod(public_door, loop_card, 20201227), powermod(public_card, loop_door, 20201227)
   
end

day25_part1 (generic function with 2 methods)

In [24]:
@btime day25_part1()

"Pub door: 2084668, card: 3704642"

"loop door 12419160, card 2115361"

"Pub door: 2084668, card: 3704642"

"loop door 12419160, card 2115361"

"Pub door: 2084668, card: 3704642"

...

"loop door 12419160, card 2115361"

"Pub door: 2084668, card: 3704642"

"loop door 12419160, card 2115361"

  4.485 s (371 allocations: 28.58 KiB)


(42668, 42668)